<a href="https://colab.research.google.com/github/marcin-mulawa/deep_learning-cdv/blob/main/6.%20dense_sentiment_classifier/6.%20dense_sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dense Sentiment Classifier

#### Zaprojektujmy gęstą sieć do klasyfikacji sentymentu (nastroju / nastawienia) recenzji filmów z portalu IMDB

#### Załadujmy zależności (zwróć uwagę na nowy rodzaj warstwy - Embedding)

In [25]:
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint 
import os
from sklearn.metrics import roc_auc_score, roc_curve, auc, RocCurveDisplay
import pandas as pd
import matplotlib.pyplot as plt

**Ustawmy stałe i hiperparametry (gdy wiemy, że przetwarzanie zbioru danych będzie bardziej złożone, warto to robić już na początku), wybierz dowolną liczbę epok i batch_size. Przypatrz się co oznaczają wszystkie hiperparametry, przyda się to w zrozumieniu całego procesu przetwarzania**

In [26]:
output_dir = 'output' #sieć będzie generować wyjście, więc określamy jego lokalizację

#hiperparametry treningu
epochs = 50
batch_size = 512

# hiperparametry przetwarzania tekstu: 
n_dim = 64 #liczba wymiarów przestrzeni wektorów słów
n_unique_words = 5000 # liczba uwzględnianych najpopularniejszych słów w korpusie recenzji filmów
n_words_to_skip = 50 # liczba najczęstszych stop words do usunięcia
max_review_length = 100 # długość recenzji, dłuższe będą przycinane, a krótsze sztucznie wydłużane (tak jakby padding) 
pad_type = trunc_type = 'pre' #przycinanie i wydłużanie będzie miało miejsce na początku recenzji (alternatywa do 'post')

# hiperparametry sieci: 
n_dense = 64
dropout = 0.5

#### Załadujmy dane

Zbiór składa się z 50 000 recenzji - połowa w zbiorze treningowym, połowa w walidacyjnym. Wraz z recenzją dodawane są liczby gwiazdek jako oceną od 1 do 10. Recenzję ocenioną na <=4 gwiazdki uznaje się za negatywną (y=0), a na >=7 gwiazdek za pozytywną (y=1), neutralnych recenzji nie ma w zbiorze.

W metodzie `load_data` kryje się już kilka metod automatycznego przetwarzania tekstu:
 - tokenizacja
 - usuwanie znaków interpunkcyjnych
 - zamiana na małe litery
 - zamiana słów na całkowitoliczbowe indeksy
 - ograniczenie wielkości słownika (num_words)
 - usunięcie najpopularniejszych stop words (skip_top)
 - brakuje w niej ewentualnego stemmingu i lematyzacji oraz analizy n-gramów

In [3]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words, skip_top=n_words_to_skip) 

17473536/17464789 [==============================] - 0s 0us/step


#### Wyświetlmy pierwsze recenzje ze zbioru treningowego

Tokeny są reprezentowane przez całkowitoliczbowe indeksy, posortowane wg częstości występowania. Pierwsze kilka liczb ma specjalne znaczenie:
 - 0 - token dopełniający
 - 1 - token startowy (początek recenzji), ale tutaj jest to token nieznany gdyż token startowy znajduje się wśród 50 najpopularniejszych tokenów
 - 2 - token występujący bardzo często (czyli usunięty jako stop word) lub bardzo rzadko (czyli nieuwzględniony), zastąpiony w związku z tym tokenem 'nieznany' (UNK)
 - 3 - słowo które występuje w korpusie najczęściej
 - 4 - drugie najczęściej występujące słowo
 - 5 - trzecie najczęściej występujące słowo itd.

In [4]:
x_train[0:6]

array([list([2, 2, 2, 2, 2, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 2, 173, 2, 256, 2, 2, 100, 2, 838, 112, 50, 670, 2, 2, 2, 480, 284, 2, 150, 2, 172, 112, 167, 2, 336, 385, 2, 2, 172, 4536, 1111, 2, 546, 2, 2, 447, 2, 192, 50, 2, 2, 147, 2025, 2, 2, 2, 2, 1920, 4613, 469, 2, 2, 71, 87, 2, 2, 2, 530, 2, 76, 2, 2, 1247, 2, 2, 2, 515, 2, 2, 2, 626, 2, 2, 2, 62, 386, 2, 2, 316, 2, 106, 2, 2, 2223, 2, 2, 480, 66, 3785, 2, 2, 130, 2, 2, 2, 619, 2, 2, 124, 51, 2, 135, 2, 2, 1415, 2, 2, 2, 2, 215, 2, 77, 52, 2, 2, 407, 2, 82, 2, 2, 2, 107, 117, 2, 2, 256, 2, 2, 2, 3766, 2, 723, 2, 71, 2, 530, 476, 2, 400, 317, 2, 2, 2, 2, 1029, 2, 104, 88, 2, 381, 2, 297, 98, 2, 2071, 56, 2, 141, 2, 194, 2, 2, 2, 226, 2, 2, 134, 476, 2, 480, 2, 144, 2, 2, 2, 51, 2, 2, 224, 92, 2, 104, 2, 226, 65, 2, 2, 1334, 88, 2, 2, 283, 2, 2, 4472, 113, 103, 2, 2, 2, 2, 2, 178, 2]),
       list([2, 194, 1153, 194, 2, 78, 228, 2, 2, 1463, 4369, 2, 134, 2, 2, 715, 2, 118, 1634, 2, 394, 2, 2, 119, 954, 189, 102, 2, 20

#### Wyświetlmy długość pierwszych recenzji (liczbę tokenów w nich) oraz ich sentyment

In [5]:
for x in x_train[0:6]:
    print(len(x))

218
189
141
550
147
43


In [6]:
y_train[0:6]

array([1, 0, 0, 1, 0, 0])

#### Sprawdźmy jak de facto wyglądają słowa, kryjące się pod indeksami

In [7]:
word_index = tensorflow.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["PAD"] = 0
word_index["START"] = 1
word_index["UNK"] = 2

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
word_index

**Zadanie 0. Stwórz odwrotność powyższego słownika, by móc odczytywać recenzje**

In [13]:
index_word = {v:k for k,v in word_index.items()}

#### Odczytajmy pierwszą z recenzji, indeksowo, a potem słownie

In [ ]:
x_train[0]

In [15]:
' '.join(index_word[id] for id in x_train[0])

"UNK UNK UNK UNK UNK brilliant casting location scenery story direction everyone's really suited UNK part UNK played UNK UNK could UNK imagine being there robert UNK UNK UNK amazing actor UNK now UNK same being director UNK father came UNK UNK same scottish island UNK myself UNK UNK loved UNK fact there UNK UNK real connection UNK UNK UNK UNK witty remarks throughout UNK UNK were great UNK UNK UNK brilliant UNK much UNK UNK bought UNK UNK UNK soon UNK UNK UNK released UNK UNK UNK would recommend UNK UNK everyone UNK watch UNK UNK fly UNK UNK amazing really cried UNK UNK end UNK UNK UNK sad UNK UNK know what UNK say UNK UNK cry UNK UNK UNK UNK must UNK been good UNK UNK definitely UNK also UNK UNK UNK two little UNK UNK played UNK UNK UNK norman UNK paul UNK were UNK brilliant children UNK often left UNK UNK UNK UNK list UNK think because UNK stars UNK play them UNK grown up UNK such UNK big UNK UNK UNK whole UNK UNK these children UNK amazing UNK should UNK UNK UNK what UNK UNK done do

#### Porównajmy sobie to z oryginalną recenzją. W tym celu załadujemy pełną bazę danych recenzji

In [16]:
(all_x_train,_),(all_x_valid,_) = imdb.load_data() 

In [17]:
' '.join(index_word[id] for id in all_x_train[0])

"START this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shou

#### Przetwórzmy dane poprzez ujednolicenie wielkości danych wejściowych (dopełnienie albo przycięcie)

In [18]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
x_train[0:6]

#### Sprawdźmy czy to się udało i jak w praktyce wygląda to ujednolicenie

In [20]:
for x in x_train[0:6]:
    print(len(x))

100
100
100
100
100
100


In [21]:
' '.join(index_word[id] for id in x_train[0])

"cry UNK UNK UNK UNK must UNK been good UNK UNK definitely UNK also UNK UNK UNK two little UNK UNK played UNK UNK UNK norman UNK paul UNK were UNK brilliant children UNK often left UNK UNK UNK UNK list UNK think because UNK stars UNK play them UNK grown up UNK such UNK big UNK UNK UNK whole UNK UNK these children UNK amazing UNK should UNK UNK UNK what UNK UNK done don't UNK think UNK whole story UNK UNK lovely because UNK UNK true UNK UNK someone's life after UNK UNK UNK UNK UNK us UNK"

In [22]:
' '.join(index_word[id] for id in x_train[5])

'PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD UNK begins better than UNK ends funny UNK UNK russian UNK crew UNK UNK other actors UNK UNK those scenes where documentary shots UNK UNK spoiler part UNK message UNK UNK contrary UNK UNK whole story UNK UNK does UNK UNK UNK UNK'

#### Zaprojektujmy sieć jednokierunkową do przetworzenia recenzji

**Zadanie 1.  Zaprojektuj sieć o następujących warstwach:**

    - embedding, z liczbą unikalnych słów, liczbą wymiarów przestrzeni wektorów i maksymalną długością recenzji jako parametrami (sprawdź w dokumentacji tf)
    - warstwa spłaszczająca
    - dwie warstwy gęste ReLU z dodanymi odpowiednio dropoutami
    - warstwę klasyfikującą recenzje na pozytywną i negatywną

In [27]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Dropout

model = Sequential([
    Embedding(n_unique_words, n_dim, input_length=max_review_length),
    Flatten(),
    Dense(n_dense, activation='relu'),
    Dropout(dropout),
    Dense(n_dense, activation='relu'),
    Dropout(dropout),
    Dense(1, activation='sigmoid')
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           320000    
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 64)                409664    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6

**Zadanie 2. Z czego wynikają liczby parametrów warstwy embedding i pierwszej warstwy gęstej?**

Parametry warstwy embedding wynikają z liczby unikalnych słów w zbiorze, wielkości przestrzeni wektorów na wyjściu jaką chcemy uzyskać, maksymalnej długosći recenzji

#### Skonfigurujmy model

**Zadanie 3. Skompiluj model z odpowiednimi dla tego problemu funkcją kosztu i metryką oraz dowolnym optymalizatorem**

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Stwórzmy obiekt i katalog do rejestrowania wag modelu

In [30]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

**Zadanie 4. Naucz sieć na zbiorze treningowym z wyznaczonymi na wstępie hiperparametrami, ze zbiorem walidacyjnym, z użyciem rejestratora wag (sprawdź w dokumentacji tf)**

In [ ]:

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

**Zadanie 5. Załaduj wagi z ostatniej epoki (sprawdź w dokumentacji tf) i dokonaj ewaluacji (inferecji) modelu na zbiorze walidacyjnym**

In [ ]:
# Load the weights from the last epoch and perform model evaluation (inferencing) on the validation set
model.load_weights(output_dir+"/weights.{epoch:02d}.hdf5".format(epoch=epochs))



In [ ]:
y_pred = model.predict(x_valid)

In [ ]:
len(y_pred)

In [ ]:
y_pred[0]

In [ ]:
y_valid[0]

#### Wyświetlmy histogram dla danych walidacyjnych

In [ ]:
plt.hist(y_pred)
_ = plt.axvline(x=0.5, color='orange')

Co widzimy? Model jest najczęściej bardzo zdecydowany, większość wartości znajduje się w skrajnych częściach wykresu. Pomarańczowa linia oznacza próg 0.5, powyżej którego model po prostu ocenia recenzję jako pozytywną. Spróbujmy jeszcze zatem na różne sposoby zbadać skuteczność klasyfikacji binarnej.

In [ ]:
roc_auc_score(y_valid, y_pred)*100.0

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Model')
display.plot()
plt.show()

Krzywa roc wizualizuje zależność między skutecznością klasyfikacją próbek pozytywnych a nietrafnością klasyfikacji przypadków negatywnych (np. podział klientów banku na spłacających i niespłacających kredyty) 

#### Zgromadźmy wyniki, np. w postaci DataFrame'a,

In [ ]:
float_y_pred = []
for y in y_pred:
    float_y_pred.append(y[0])
ydf = pd.DataFrame(list(zip(float_y_pred, y_valid)), columns=['y_pred', 'y'])
ydf.head(10)

**Zadanie 6. Wyświetl tę recenzję, która została błędnie zaklasyfikowana z największą nieprawidłową pewnością modelu (zauważ i weź pod uwagę, że jeśli wartość prawdopodobieństwa jest bardzo mała, to model również ma dużą pewność - że recenzja jest negatywna)**

In [ ]:
#tu odpowiedź